In [ ]:
import numpy as np
import pandas as pd
import csv
import matplotlib.pyplot as plt
import plotly.express as px
from scipy.fftpack import fft
import plotly.express as px

import os

: 

In [65]:
# Função de criação do espectro de FFT

def espectro_fft(x, Fs, pl = False):
    n = np.size(x)
    fr = (Fs/2)*np.linspace(0,1,int(n/2))
    # Compute FFT
    X = fft(x)
    X_m = (2/n)*abs(X[0:np.size(fr)])
    #print(X_m)
    #print(fr)
    if (pl == True):
        plt.plot(fr,X_m); plt.title('Magnitude Spectrum')
        title = r"Aceleração($m/s^2$)"
        plt.xlabel('Frequency(Hz)'); plt.ylabel(title)
        plt.tight_layout()
        plt.grid('on')
        plt.show()
    return fr, X_m

|#  | Condição      | Engrenamento | Rotor | Velocidade | Número da medição|
|0  | Desligado     | Não          | Não   | 0          | mpu6050c19.csv|
|1  | Aberto        | Não          | Não   | v3         | mpu6050c20.csv|
|2  | Aberto        | Não          | Não   | v2         | mpu6050c21.csv|
|3  | Aberto        | Não          | Não   | v1         | mpu6050c22.csv|
|4  | Desbal        | Não          | Não   | v3         | mpu6050c23.csv|
|5  | Desbal        | Não          | Não   | v2         | mpu6050c24.csv|
|6  | Desbal        | Não          | Não   | v1         | mpu6050c25.csv|
|7  | Aberto        | Fixo         | Não   | v3         | mpu6050c26.csv|
|8  | Aberto        | Giro         | Não   | v3         | mpu6050c27.csv|
|9  | Aberto        | Fixo         | Sim   | v3         | mpu6050c28.csv|
|10 | BPF           | Fixo         | Sim   | v3         | mpu6050c29.csv|
|11 | Desbal1       | Fixo         | Sim   | v3         | mpu6050c30.csv|
|12 | Desbal1+BPF   | Fixo         | Sim   | v3         | mpu6050c31.csv|
|13 | Desbal2       | Fixo         | Sim   | v3         | mpu6050c32.csv|
|14 | Aberto        | Giro         | Sim   | v3         | mpu6050c33.csv|

In [66]:
df= pd.read_csv(r"./Maquinas/Ventilador_3_velocidades/mpu6050c22.csv")

In [67]:
df

,micros,ax,ay,az
0,0,-8300,-4996,-1256
1,1000,-8164,-4960,-920
2,2000,-9024,-8436,-1232
3,3000,-10872,-13620,-2304
4,4000,-12388,-19412,-3392
...,...,...,...,...
9995,9995000,-12764,-20336,-4004
9996,9996000,-11364,-16584,-3284
9997,9997000,-9252,-11284,-2244
9998,9998000,-7360,-6572,-1580


In [68]:
df['micros'] = df['micros']/1e6

In [69]:
df = df.set_index('micros')

In [70]:
# Conversão para g
df = df * (9.81 / 16384.0)
df

,ax,ay,az
micros,,,
0.000,-4.969666,-2.991379,-0.752036
0.001,-4.888235,-2.969824,-0.550854
0.002,-5.403164,-5.051096,-0.737666
0.003,-6.509663,-8.155042,-1.379531
0.004,-7.417375,-11.623030,-2.030977
...,...,...,...
9.995,-7.642507,-12.176279,-2.397415
9.996,-6.804250,-9.929751,-1.966311
9.997,-5.539680,-6.756350,-1.343606


In [71]:
df.ax = df.ax.values - np.mean(df.ax.values)
df.ay = df.ay.values - np.mean(df.ay.values)
df.az = df.az.values - np.mean(df.az.values)
df

,ax,ay,az
micros,,,
0.000,1.007039,4.599913,0.726720
0.001,1.088470,4.621468,0.927902
0.002,0.573541,2.540196,0.741090
0.003,-0.532958,-0.563749,0.099225
0.004,-1.440671,-4.031738,-0.552220
...,...,...,...
9.995,-1.665802,-4.584987,-0.918658
9.996,-0.827546,-2.338459,-0.487555
9.997,0.437025,0.834942,0.135150


In [72]:
# calculando o dt
tam = df.shape[0]
dt_vec = (df.index[1:tam]-df.index[0:(tam-1)])
df_dt = pd.DataFrame(dt_vec)
#print(dt_vec[dt_vec>1])
fig = px.line(df_dt)
fig.show()

In [73]:
fs = 1/np.mean(dt_vec)
print(f"Frequência de amostragem = {fs} Hz")

Frequência de amostragem = 1000.0 Hz


In [74]:
f, A  = espectro_fft(df.az.values, fs)

In [75]:
df_freq = pd.DataFrame(np.transpose([f, A]), columns=['Freq', 'Amp'])
df_freq

,Freq,Amp
0,0.00000,7.780443e-17
1,0.10002,2.239985e-03
2,0.20004,1.316865e-03
3,0.30006,9.469755e-04
4,0.40008,6.797960e-04
...,...,...
4995,499.59992,7.411389e-04
4996,499.69994,4.576316e-04
4997,499.79996,3.802981e-04
4998,499.89998,5.555934e-04


In [76]:
fig = px.line(df, x = df.index, y = 'az')
fig.show()

In [77]:
fig = px.line(df_freq, x = 'Freq', y = 'Amp')
fig.show()